<a href="https://colab.research.google.com/github/janani1701/ChatBotUsingGPT/blob/main/ChatBotUsingGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###Installing required libraries


In [ ]:
!pip install openai
!pip install pandas
!pip install langchain
!pip install pandasai
!pip install langchain_experimental
!pip install flask flask-ngrok pandas pyngrok


  Using cached langchain_experimental-0.0.57-py3-none-any.whl (193 kB)


###Importing necessary libraries


In [ ]:
import openai
import pandas as pd
from pandasai import SmartDataframe
import pandas as pd
import openai
from pandasai.llm.openai import OpenAI
from langchain.llms import OpenAI
from pandasai import Agent
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
from flask import Flask, render_template, request, jsonify
from flask_ngrok import run_with_ngrok
from flask import Flask
from langchain_experimental.agents.agent_toolkits.pandas.prompt import PREFIX
import time


###Uploading Financials.csv

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Financials.csv to Financials.csv


###Setting OpenAI API key


In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'yourApiKey'
llm = OpenAI()


###Reading Financials.csv file into a DataFrame


In [ ]:
df =  pd.read_csv(list(uploaded.keys())[0])


###Cleaning DataFrame columns and converting data types


In [ ]:
df.columns = df.columns.str.strip()
df['Gross Sales'] = df['Gross Sales'].str.strip().str.replace("$",'').str.replace(",",'').astype('float')
df['Units Sold'] = df['Units Sold'].str.strip().str.replace("$",'').str.replace(",",'').astype('float')
df['Manufacturing Price'] = df['Manufacturing Price'].str.strip().str.replace("$",'').str.replace(",",'').astype('float')
df['Sale Price'] = df['Sale Price'].str.strip().str.replace("$",'').str.replace(",",'').astype('float')
df['Sales'] = df['Sales'].str.strip().str.replace("$",'').str.replace(",",'').astype('float')
df['COGS'] = df['COGS'].str.strip().str.replace("$",'').str.replace(",",'').astype('float')
df['Discounts'] = df['Discounts'].str.strip().str.replace("$",'').str.replace(",",'').str.replace("-",'0').astype('float')
df['Profit'] = df['Profit'].str.strip().str.replace("$",'').str.replace(",",'').str.replace("-",'0').str.replace("(",'').str.replace(")",'').astype('float')


<ipython-input-9-555a15ac9436>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Gross Sales'] = df['Gross Sales'].str.strip().str.replace("$",'').str.replace(",",'').astype('float')
<ipython-input-9-555a15ac9436>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Units Sold'] = df['Units Sold'].str.strip().str.replace("$",'').str.replace(",",'').astype('float')
<ipython-input-9-555a15ac9436>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Manufacturing Price'] = df['Manufacturing Price'].str.strip().str.replac

###Uploading Instructions.txt file

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Instructions.txt to Instructions.txt


###Reading Instructions.txt file


In [ ]:
instruction_file_name = list(uploaded.keys())[0]
with open(instruction_file_name, 'r') as file:
    instructions = file.read()
print(instructions)



Segment Reporting: If a query asks about Government, respond with, "I cannot disclose details related to government."

Sales and Profit Correlation: When sales data is requested, mention, "I can provide trends and correlations between sales and profit without disclosing exact gross sales figures."

Germany Profit Reporting: If a query asks about Germany profit, respond with, ""I can provide insights into profit trends and margins without disclosing exact profit figures."

Montana Gross Sales: For inquiries related to gross sales of the product Montana, "I can provide insights into its performance trends and correlations with other metrics, without disclosing exact gross sales figures."

Discounts Reporting: If a query asks about discounts, respond with, "While I cannot provide specific discount figures, I can confirm whether a discount was applied."

Yearly Units Sold: When there's a query about units sold in a year for France, respond with, "Detailed regional data is sensitive. Summar

###Creating a prefix for the agent

In [ ]:
PREFIX = instructions

###Creating a Pandas DataFrame agent


In [ ]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=openai.api_key),
    df,
    verbose=True,
    prefix = PREFIX,
    agent_type=AgentType.OPENAI_FUNCTIONS
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


###Printing the port number to which the Flask application is being proxied in Google Colab


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))


https://6i71vpi6648-496ff2e9c6d22116-5000-colab.googleusercontent.com/


###Setting up Flask application


In [ ]:
app = Flask(__name__)



###Endpoint for processing queries


In [ ]:
app = Flask(__name__)
@app.route('/query', methods=['POST'])
def process_query():
    if request.method == 'POST':
        start_time = time.time()
        query = request.json.get('query')
        if query and agent:
            response = agent.run(query)
            end_time = time.time()
            process_time = end_time - start_time
            return jsonify({'response': response, 'process_time' : process_time})
        else:
            return jsonify({'error': 'No query provided'})



###Main page with HTML form for submitting queries


In [ ]:
@app.route('/')
def index():
    index_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Financial Analysis</title>
</head>
<body>
    <h1>Financial Analysis</h1>


    <hr>

    <h2>Query</h2>
    <textarea id="query" rows="4" cols="50" placeholder="Enter your query..."></textarea>
    <button onclick="sendQuery()">Submit Query</button>
    <button onclick="clearQuery()">Clear Query</button>

    <hr>

    <h2>Response</h2>
    <div id="response"></div>

    <h2>ProcessTime</h2>
    <div id = "process-time"> </div>



    <script>
        function sendQuery() {
            var query = document.getElementById("query").value;
            fetch("/query", {
                method: "POST",
                headers: {
                    "Content-Type": "application/json"
                },
                body: JSON.stringify({ query: query })
            })
            .then(response => response.json())
            .then(data => {
                document.getElementById("response").innerText = data.response;
                document.getElementById("process-time").innerText = data.process_time + " seconds";
            })
            .catch(error => console.error('Error:', error));
        }
        function clearQuery() {
            document.getElementById("query").value = "";
            document.getElementById("response").innerText = "";
            document.getElementById("process-time").innerText = "";
        }
    </script>
</body>
</html>
"""
    return index_html


###Running the Flask application


In [ ]:
if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/May/2024 03:18:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/May/2024 03:18:26] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Year'].iloc[0]"}`


2014

INFO:werkzeug:127.0.0.1 - - [02/May/2024 03:18:46] "POST /query HTTP/1.1" 200 -


The first value under the column "Year" is 2014.

> Finished chain.
